## Step 1 - Preparing data for json file

In [1]:
#Preparing data to save in a Json File
import pandas as pd
import json

artifacts_df = pd.read_csv(open("../data/selected-artifacts.csv", encoding='utf8'))
artifacts_df.set_index('id', inplace=True)
studies_df = pd.read_csv(open("../data/selected-studies-with-abstract-and-type.csv", encoding='utf8'), index_col='id')

for i in artifacts_df.index:

  artifacts_df['td_type'][i] = artifacts_df['td_type'][i].replace(' TD','').lower().replace('defect','').replace('security','').lower().replace(" ", "").replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['tdma'][i] = artifacts_df['tdma'][i].lower().replace(" ", "").replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['output_info'][i] = str(artifacts_df['output_info'][i]).lower().replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['output_fmt'][i] = artifacts_df['output_fmt'][i].lower().replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['input_info'][i] = artifacts_df['input_info'][i].lower().replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['input_fmt'][i] = artifacts_df['input_fmt'][i].lower().replace(" ", "").replace("[", "").replace("]","").replace("'", "").split(',')
  #artifacts_df['programming_language'][i] = artifacts_df['programming_language'][i].lower().split(',')
  #artifacts_df['license_type'][i] = str(artifacts_df['license_type'][i]).lower().split(',')
  artifacts_df['studies_citing_artifact'][i] = artifacts_df['studies_citing_artifact'][i].replace("[", "").replace("]","").replace("'", "").split(',')
  artifacts_df['other_comments'][i] = str(artifacts_df['other_comments'][i]).split(',')
  artifacts_df['is-integrated-description'][i] = str(artifacts_df['is-integrated-description'][i]).split('\n')
  artifacts_df['is-integrated-with'][i] = str(artifacts_df['is-integrated-with'][i]).split('\n')
  artifacts_df['is-integrated-using-interface-subtype'][i] = str(artifacts_df['is-integrated-using-interface-subtype'][i]).split('\n')
  artifacts_df['interface-subtype'][i] = str(artifacts_df['interface-subtype'][i]).split('\n')
  artifacts_df['trigger-description'][i] = str(artifacts_df['trigger-description'][i]).split('\n')
  #artifacts_df['devlopment-activity-mapping'][i] = str(artifacts_df['devlopment-activity-mapping'][i])

In [4]:
studies_df = studies_df.drop("Unnamed: 0", axis=1)
studies_df

,title,abstract,study-type,study-type-description,author,author_type,year,venue,venue_type,link,reference,objective,artifacts
id,,,,,,,,,,,,,
S1,CCFinder: A Multilinguistic Token-Based Code C...,A code clone is a code portion in source files...,propose,NaN,"Toshihiro Kamiya, Member\nShinji Kusumoto\nKat...",Academia,"2,002.00",IEEE TRANSACTIONS ON SOFTWARE ENGINEERING,Journal,NaN,"@article{Kamiya2002CCFinderAM,\n title={CCFin...",NaN,CCFinderX
S2,A Catalogue of Lightweight Visualizations to S...,Preserving the integrity of software systems i...,use,use in an empirical study,Chris Parnin\nCarsten Gorg\nOgechi Nnadi,Academia,"2,008.00",ACM symposium on Software visualization,Conference,https://doi.org/10.1145/1409720.1409733,"@INPROCEEDINGS{Parnin2008-mo,\n title ...",NaN,Gendarme
S3,Metrics Functions for Kanban Guards,Agile and lean approaches favor self-organizin...,point,the study points to tools as examples,Jeanette Heidenberg\nIvan Porres,Academia,"2,010.00",International Conference and Workshops on Engi...,Conference,http://doi.org/10.1109/ECBS.2010.43,"@inproceedings{Heidenberg2010,\n doi = {10.11...","In this paper, we outline metrics functions th...",STAN\nResource standard metrics
S4,Building Empirical Support for Automated Code ...,Identifying refactoring opportunities in softw...,propose,NaN,Jan Schumacher\nNico Zazworka\nForrest Shull\n...,Academia,"2,010.00",International Symposium on Empirical Software ...,Conference,NaN,"@inproceedings{10.1145/1852786.1852797,\nautho...",NaN,CodeVizard
S5,The SQALE Quality and Analysis Models for Asse...,This article presents the quality and analysis...,propose,NaN,Thierry Coq\nJean-Pierre Rosen,Industry,"2,011.00",International Conference on Reliable Software ...,Conference,https://doi.org/10.1007/978-3-642-21338-0_5,"@incollection{Coq_2011,\n\tdoi = {10.1007/978-...",This article presents the quality and analysis...,AdaControl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
S174,Self-admitted technical debt detection by lear...,The goal of software development is to deliver...,propose,NaN,NaN,Academia,"2,022.00",Journal of Software: Practice and Experience,Journal,NaN,"@article{Li_2022,\n doi = {10.1002/spe....",NaN,NaN
S175,Sonarlizer Xplorer: a tool to mine Github proj...,The advancement of artificial intelligence and...,propose,NaN,NaN,Academia,"2,022.00",International Conference on Technical Debt,Conference,NaN,"@INPROCEEDINGS{9804475, author={Pina, Diogo a...",NaN,NaN
S176,Merging Smell Detectors: Evidence on the Agree...,Technical Debt estimation relies heavily on th...,compare,NaN,NaN,Academia,"2,022.00",International Conference on Technical Debt,Conference,NaN,"@INPROCEEDINGS{9804506, author={Ichtsis, Apos...",NaN,NaN


## Step 2- Retrieving artifact's first citation year

In [5]:
artifacts_df['first_citation'] = '-'
for i in artifacts_df.index:
    years = set([studies_df['year'][x] for x in studies_df.index if str(x) in artifacts_df['studies_citing_artifact'][i]])
    artifacts_df['first_citation'][i] = min(years)

In [7]:
list_studies_comparing = [i for i in studies_df.loc[studies_df['study-type'] == 'compare'].index]
list_studies_comparing

['S6',
 'S20',
 'S29',
 'S31',
 'S34',
 'S35',
 'S38',
 'S39',
 'S74',
 'S105',
 'S129',
 'S145',
 'S146',
 'S172',
 'S176']

## Step 3 - Retrieving studies that compared the artifact

In [8]:
artifacts_df['studies-compared-artifact'] = '-'
list_studies_comparing = [i for i in studies_df.loc[studies_df['study-type'] == 'compare'].index]
list_artifacts_compared = set([i for x in list_studies_comparing for i in artifacts_df.index if x in artifacts_df['studies_citing_artifact'][i]])
for i in artifacts_df.index:
    studies_compared_artifact = []
    for x in list_studies_comparing:
        if x in artifacts_df['studies_citing_artifact'][i]:
            studies_compared_artifact.append(x)
    artifacts_df['studies-compared-artifact'][i] = studies_compared_artifact

## Step 4- Storing Json file

In [9]:
#Storing dataset in a JSON file

result = artifacts_df.to_json(orient='index')
json_raw = json.loads(result)
with open('../data/studies-and-artifacts.json', 'w') as f:
    json.dump(json_raw,f)